# David Ryan's ANTS methond
- 20-25 % increase in price for 15 days
- 20-25 % increase in volume for 15 days
- 12 out of 15 days up

In [ ]:
# The goal is to determine if the average volume over the 15 day range is greater than the 50-day average volume before the range.
# A couple of things to consider. Your range shows 12 days, all calculations are based on using all 15 days in the range.
#Also, the % change is not simply the move from the start to the end of the range, 
#it's calculated as the average change over the range. If you take all 15 closes, 
#get the average and compare to close before the range, the % change is < 20%
# https://www.tradingview.com/script/uEaFv7tm-Ants-Momentum-Volume-and-Price-MVP/

In [ ]:
import os
import pandas as pd

In [ ]:
for dirs in ['mega']:
    main_root=f'/home/thakur/test/{dirs}'
    stocks=os.listdir(main_root)
    #stocks=os.listdir(f'/home/thakur/test/{dirs}')
    print(stocks)

In [ ]:
for k,i in enumerate(stocks):
    if k!=0:continue
    print(f'{k}')
    file=f'{main_root}/{i}'
    print(f'{file}')

In [ ]:
file

In [ ]:
df=pd.read_csv(file)

In [ ]:
def get_sub_df(df,begin=0):
    '''
    returns two dfs temp_head and temp_tail
    '''
    temp_df=df.tail(65+begin)
    temp_df=temp_df.head(65)
    temp_head_vol=temp_df.head(50).Volume.mean() #for average volume
    temp_tail=temp_df.tail(16) #for price and volume
    return temp_head_vol,temp_tail
    

In [ ]:
def get_15_day_status(temp_df):
    #up for at least 12 out of 15 days
    temp_test=temp_df['Adj Close']-temp_df.shift(1)['Adj Close']
    return sum(temp_test.tail(15)>0)>11

In [ ]:
def get_volume_status(temp_df,vol):
    return temp_df.tail(15).Volume.mean()>1.20*vol

In [ ]:
def get_increment_status(temp_df):
    first=temp_df.head(1)['Adj Close'].values[0]
    last=temp_df.tail(1)['Adj Close'].values[0]
    #print(f'first: {first}')
    #print(f'last : {last}')
    
    increments=(last-first)*100/first
    #print(f'increments: {increments}')
    return increments>20.0 and increments<25.0
    

In [ ]:

def check_all_in_dir(dirs):
    main_root=f'/home/thakur/test/{dirs}'
    print(f'Working Dir: {main_root}')
    stocks=os.listdir(main_root)
    #stocks=os.listdir(f'/home/thakur/test/{dirs}')
    ticker=[];days_up=[];volume_up=[];price_up=[]
    #print(stocks)
    for i in stocks:
        file=f'{main_root}/{i}'
        #print(f'{file}')
        df=pd.read_csv(file)
        #50 day mean volume and tail df
        vol,tail=get_sub_df(df)
        
        day_up=get_15_day_status(tail)
        tail=tail.tail(15)
        vol_up=get_volume_status(tail,vol)
        increment_status=get_increment_status(tail)
        
        ticker.append(i.split('.')[0])
        days_up.append(day_up)
        volume_up.append(vol_up)
        price_up.append(increment_status)
        
    zipped=list(zip(ticker,days_up,volume_up,price_up))
    df = pd.DataFrame(zipped, columns=['TICKER', 'DAYS_UP', 'VOLUME_UP','INCREMENT']).reset_index(drop=True)

    return df
        
    

In [ ]:
my_list=['mega','medium','large','small','micro','nano']
condition='DAYS_UP & VOLUME_UP & INCREMENT'
for i,k in enumerate(my_list,start=1):
    print(40*'=')
    print(f"\n{i}/{len(my_list)} Working With {k.upper()}..\n")
    
    df_m=check_all_in_dir(k)
   
    df_tem=df_m.query(condition).reset_index(drop=True)
    
    if not df_tem.empty:
        print(f'Saved into the file: david_{k}.csv\n')
        df_tem.to_csv(f'david_{k}.csv',index=False)
        print(df_tem)
    